# 学习笔记
严格来说，这个是在学习《实战Google深度学习框架》时记录并实践一些局部的知识点或者一些小技巧，所以可能不同部分之间没有明显联系，都是独立从章节中抽取出来。相对完整的章节内容，都按照章节或者模块内容区分，放在同一个文件中。

## 自定义损失函数
在机器学期过程中，我们常常会使用损失函数（cost function)来优化模型，以满足实际需求。除了经典的损失函数外，有时候我们需要根据实际情况定义一些损失函数。

比如说，在预测商品价格时，预测多了，商家损失的是生产商品的生产成本；预测少了，损失的是商品的利润。考虑到商品的成本和利润不会严格相等，假如一个商品的成本是1元，但利润是10元，当我们使用如**均方误差**这样的损失函数时，有可能无法实现利润最大化。所以，此时我们需要定义一个损失函数，该损失函数需要与利润直接联系起来。

In [ ]:
# 定义损失函数如下：
"""
loss = tf.reduce_sum(tf.where(tf.greater(v1, v2),
                               (v1 - v2) * a, (v2 - v1) * b))
"""
# 当v1大于v2时，执行（v1 - v2) * a；当v1不大于v2时，执行（v2 - v1) * b


In [ ]:
# tf.where和tf.greater的实例
import tensorflow as tf
v1 = tf.constant([1.0, 2.0, 3.0, 4.0])
v2 = tf.constant([4.0, 3.0, 2.0, 1.0])

sess = tf.InteractiveSession()
print tf.greater(v1, v2).eval()
# 输出[False False True True]

print tf.where(tf.greater(v1, v2), v1, v2).eval()
# 输出[4. 3. 3. 4.]

sess.close()

下面用一个简单的神经网络实现上面的这个自定义函数
* 两层的神经网络
* 两个输入节点，一个输出节点
* 数据集随机生成

In [ ]:
import tensorflow as tf
from numpy.random import RandomState

batch_size = 8

x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

w1 = tf.Variable(tf.random_normal([2, 1], stddev=1, seed=1))
y = tf.matmul(x, w1)

# 预测多了和预测少了的成本
loss_less = 10
loss_more = 1

loss = tf.reduce_sum(tf.where(tf.greater(y, y_),
                               (y - y_) * loss_more, 
                               (y_ - y) * loss_less))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

# 随机生成数据集，噪声为-0.05 ～ 0.05的随机数
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = [[x1 + x2 + rdm.rand()/10.0 - 0.05] for (x1, x2) in X]

# 训练神经网络
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        sess.run(train_step, feed_dict = {x: X[start: end],
                                          y_: Y[start: end]})
        print sess.run(w1)

# 滑动平均模型（简单样例）
这是一种可以使得模型在测试数据上更健壮（robust）的方法，在采用随机梯度下降算法训练神经网络时，可以在一定程度上提升最终模型在测试数据上的表现。

使用的函数：
tf.train.ExponentialMovingAverage

每次使用的衰减率：
min{decay, (1+num_updates)/(10+num_updates)

每次更新变量的值：
shadow_variable = decay*shadow_variable + (1-decay)*variable

In [ ]:
import tensorflow as tf

v1 = tf.Variable(0, dtype=tf.float32)
step = tf.Variable(0, trainable=False)

# 定义一个滑动平均的类,衰减率为0.99, 控制衰减率的变量step
ema = tf.train.ExponentialMovingAverage(0.99, step)

# 定义一个更新滑动平均的操作
maintain_averages_op = ema.apply([v1])

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print sess.run([v1, ema.average(v1)])
    
    # 更新v1的值为5,更新结果为4.5
    sess.run(tf.assign(v1, 5))
    sess.run(maintain_averages_op)
    print sess.run([v1, ema.average(v1)])
    
    # 更新step的值为10000
    sess.run(tf.assign(step, 10000))
    # 更新v1的值为10,更新结果为4.555
    sess.run(tf.assign(v1, 10))
    sess.run(maintain_averages_op)
    print sess.run([v1, ema.average(v1)])
    
    # 再次更新，结果为4.60945
    sess.run(maintain_averages_op)
    print sess.run([v1, ema.average(v1)])

# 模型持久化
Tensorflow提供了tf.train.Saver类用来保存和还原一个神经网络模型

In [ ]:
# 保存模型
import tensorflow as tf

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='v1')
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name='v2')
result = v1 + v2

init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, "./saver/model_eg.ckpt")

In [ ]:
# 加载计算图，不用重复图中的计算流程
import tensorflow as tf

saver = tf.train.import_meta_graph("./saver/model_eg.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess, "./saver/model_eg.ckpt")
    print sess.run(tf.get_default_graph().get_tensor_by_name("add:0"))


In [ ]:
# 加载时变量重命名
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='other-v1')
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name='other-v2')

saver = tf.train.Saver({"v1":v1, "v2":v2})
print v1, v2

以上功能的主要目的之一是，为了方便使用变量的滑动平均值。以下是实现的代码：

In [ ]:
import tensorflow as tf

v = tf.Variable(0, dtype=tf.float32, name="v")
for variables in tf.global_variables():
    print variables.name
    
ema = tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op = ema.apply(tf.global_variables())
for variables in tf.global_variables():
    print variables.name
    
saver = tf.train.Saver()
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    sess.run(tf.assign(v, 10))
    sess.run(maintain_averages_op)
    
    saver.save(sess, "./saver/model.ckpt")
    print sess.run([v, ema.average(v)])

In [ ]:
# variables_to_restore函数
v = tf.Variable(0, dtype=tf.float32, name="v")
ema = tf.train.ExponentialMovingAverage(0.99)
print ema.variables_to_restore()

saver = tf.train.Saver({"v/ExpinentialMovingAverage": v})
with tf.Session() as sess:
    saver.restore(sess, "./saver/model.ckpt")
    print sess.run(v)

# TFRecord格式
TFRecord文件中的数据都是通过tf.train.Example Protocol Buffer的格式存储的。以下是tf.train.Example的定义：

In [2]:
"""
message Example {
    Features features = 1;
};

message Features {
    map<string, Feature> feature = 1;
};

message Feature {
    one of kind {
        BytesList bytes_list = 1;
        FloatList float_list = 2;
        Int64List int64_list = 3;
    }
};
"""

'\nmessage Example {\n    Features features = 1;\n};\n\nmessage Features {\n    map<string, Feature> feature = 1;\n};\n\nmessage Feature {\n    one of kind {\n        BytesList bytes_list = 1;\n        FloatList float_list = 2;\n        Int64List int64_list = 3;\n    }\n};\n'

# 图像数据处理函数

In [ ]:
"""
# 编码处理
tf.image.decode_jpeg()
tf.image.encode_jpeg()
# 大小调整
tf.image.resize_images()
# 图像翻转
tf.image.flip_up_down()
tf.image.flip_left_right()
tf.image.transpose_image()
# 图像色彩调整
tf.image.adjust_brightness()
tf.image.adjust_contrast()
tf.image.adjust_hue()
tf.image.adjust_saturation()
# 处理标注框
tf.image.draw_bounding_boxes()

"""